# Applied Data Science Capstone project

**Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods** 

Author: Rachid de Graaff


#Introduction 

This analysis attempts to present a dedicated exploration of the city of Rotterdam in The Netherlands. In addition, the exploration goes in pair with retrieving information of Medditerean/Moroccan restaurants. As such, with the retrieved insights one can determine the business opportunities in potential areas of the city. At least it gives a good representation where to explore the excellent cuisine of Medditerean/Moroccan food. 
In order to grasp the information adequately of the restaurants in Rotterdam, Geoplots will be the main visualization applied. In particular, the retrieved data on restaurants will be plotted on maps and an attempt to determine clusters (k-means clustering) upon areas is shown. From there, one can identify areas which are more dense with restaurants with respect to other areas. Thus, potential locations for a new restaurant can be determined accordingly.  
The target audience of the project are those who are interested in establishing/moving a Medditerean/Moroccan restaurant in an area in Rotterdam, or simply want to know where to enjoy a great meal. 

# Description of the Problem


This project aims at finding a starting point for a Medditerean/Moroccan restaurant. Specifically, this report will be targeted to stakeholders interested in opening an  restaurant in Rotterdam, The Netherlands.



#Description of the Data

Based on definition of our problem, factors that will influence our decission are:

- number of and distance to Meditterean/Moroccan restaurants in the neighborhood, if any 
- distance of neighborhood from city center

I've decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods/areas.

Following data sources will be needed to extract/generate the required information:

- centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Bing Maps API reverse geocoding
- number of Meditterean/Moroccan restaurants and their type and location in every neighborhood will be obtained using Foursquare API
- coordinates of Rotterdam center will be obtained using Bing Maps API geocoding of well known Rotterdam location (Coolsingel)



**Neighborhood Candidates** 

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 6x6 killometers centered around Rotterdam city center.

Let's first find the latitude & longitude of Rotterdam city center, using specific, well known address and Geocoder with Bing Maps API.



In [0]:
!pip install geocoder
import geocoder
key = 'AjPYTSlYCGj1MxECPvPeeVAG6wXNRUyOs4xTvkFSb7vk8OvAPThy4B9j_RFRBlBt'


def get_coordinates(address):
  #g = geocoder.arcgis(address)
  g = geocoder.bing(address, key=key)
  return g.latlng

address = "Coolsingel, Rotterdam, Netherlands"
rdam_center = get_coordinates(address)
#print('Coordinate of {}: {}').format(address, rdam_center))
print('Coordinate of {}: {}'.format(address, rdam_center))

     |████████████████████████████████| 102kB 2.4MB/s 
Coordinate of Coolsingel, Rotterdam, Netherlands: [51.92224, 4.47879]


Next, let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Coolsingel. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then those coordinates will be projected back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [0]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Rotterdam center longitude={}, latitude={}'.format(rdam_center[1], rdam_center[0]))
x, y = lonlat_to_xy(rdam_center[1], rdam_center[0])
print('Rotterdam center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Rotterdam center longitude={}, latitude={}'.format(lo, la))

     |████████████████████████████████| 10.4MB 2.8MB/s 
Coordinate transformation check
-------------------------------
Rotterdam center longitude=4.47879, latitude=51.92224
Rotterdam center UTM X=-222546.79699358006, Y=5804871.70294864
Rotterdam center longitude=4.478790000000005, latitude=51.922239999999995




Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.


In [0]:
# City center in Cartesian coordinates
rdam_center_x, rdam_center_y = lonlat_to_xy(rdam_center[1], rdam_center[0]) 

# Vertical offset for hexagonal grid cells
k = math.sqrt(3) / 2 
x_min = rdam_center_x - 6000
x_step = 600
y_min = rdam_center_y - 6000 - (int(20/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(20/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 20):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(rdam_center_x, rdam_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

361 candidate neighborhood centers generated.


Let's showcase what we have on a folium map:
The city center location and candidate neighborhood centers



In [0]:
import folium

map_rdam = folium.Map(location=rdam_center, zoom_start=12)
folium.Marker(rdam_center, popup='Coolsingel').add_to(map_rdam)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='red', fill=True, fill_color='green', fill_opacity=1).add_to(map_rdam) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_rdam)
    #folium.Marker([lat, lon]).add_to(map_rdam)
map_rdam

Now we have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~ 6km from Coolsingel. Let's now use Geocoder to get approximate addresses of those locations.

In [0]:

key = 'Key'
#!pip install geocoder
import geocoder

addresses = []
latlng = [51.921009741358645, 4.47966432595502]

def get_address(latitude, longitude, api_key):
  latlng = [latitude, longitude]
  g = geocoder.bing(latlng, method='reverse', key=api_key)
  return g.address

addr = get_address(rdam_center[0], rdam_center[1], key)

print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(rdam_center[0], rdam_center[1], addr))

Reverse geocoding check
-----------------------
Address of [51.92224, 4.47879] is: Coolsingel 65, 3012 AC Rotterdam, Netherlands


In [0]:
#retrieve the location addresses
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon, key)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Netherlands', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [0]:
addresses[150:170]

['Visserijplein, 3026 Rotterdam',
 'Ruilstraat 45A, 3023 XL Rotterdam',
 'Volmarijnstraat 98A, 3021 XV Rotterdam',
 'Mathenesserlaan 166A, 3015 CL Rotterdam',
 'Eendrachtsstraat 67, 3012 XH Rotterdam',
 'Jufferkade 57, 3011 VW Rotterdam',
 'Bolwerk, 3011 Rotterdam',
 'Antwerpse Hoofd, 3071 Rotterdam',
 'Maasboulevard, 3063 Rotterdam',
 'Honingerdijk 90, 3062 NX Rotterdam',
 'De Lairesselaan 80, 3062 PK Rotterdam',
 '3065 Rotterdam',
 'Engelbrechtweg 480, 3065 SR Rotterdam',
 "'s-Gravenweg 650, 3065 SH Rotterdam",
 'Lilian Ngoyiweg, 3065 Rotterdam',
 'Tuinlaan 96, 3111 AX Schiedam',
 'Couwenhovenstraat 3, 3113 AA Schiedam',
 'Schiedamseweg Beneden 509C, 3028 BR Rotterdam',
 'Tjalklaan, 3028 Rotterdam',
 'Mathenesserdijk 115, 3027 BG Rotterdam']

Put the addresses in a dataframe

In [0]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,3085 Rotterdam,51.869029,4.486977,-222846.796994,5.798896e+06,5983.101203
1,"Kapelburg 296, 3085 HV Rotterdam",51.869803,4.495545,-222246.796994,5.798896e+06,5983.101203
2,3084 Rotterdam,51.870908,4.455905,-224946.796994,5.799416e+06,5960.494946
3,"Kruiningenstraat 127, 3086 KS Rotterdam",51.871684,4.464472,-224346.796994,5.799416e+06,5745.215401
4,"Tiengemetensingel 34A, 3086 NR Rotterdam",51.872460,4.473040,-223746.796994,5.799416e+06,5586.367335
5,"Biezeveld 11, 3085 RD Rotterdam",51.873235,4.481609,-223146.796994,5.799416e+06,5488.852339
6,"Larenkamp 32A, 3085 GC Rotterdam",51.874009,4.490178,-222546.796994,5.799416e+06,5455.960044
7,"Diepenhorst 32A, 3085 WL Rotterdam",51.874783,4.498747,-221946.796994,5.799416e+06,5488.852339
8,"Vaanweg, 3076 Rotterdam",51.875556,4.507317,-221346.796994,5.799416e+06,5586.367335
9,"Elsschotstraat 12, 3076 EA Rotterdam",51.876329,4.515887,-220746.796994,5.799416e+06,5745.215401


In [0]:
# 
 df_locations.to_pickle('./locations.pkl')


Foursquare

Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood defined earlier.

We're interested in venues in food category,and in particular Moroccan restaurants.  Since Foursquare works with category parameter in their GET function, the code for Moroccan restaurants is used. Note that the category Mediterranean is used, since Moroccan cuisine is a subcategory.

First, we'll define a function that operates as follows:

For each center of neighborhood/area to be evaluated
  - Retrieve restaurants matching the category 
  - Convert the retrieved hits from JSON to Dataframe
  - Write the Dataframe to a CSV (this is done due to limited number of calls we can make to the Foursquare API) 

In [0]:
#we have the lats and lngs of each centered location from coolsingel so lets retrieve the surrounding 
#restaurants for each location

import requests
import pandas
from pandas.io.json import json_normalize

#Function to retrieve restaurants for an area
def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=350, limit=100):
    version = '20200401'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    dataloc = json_normalize(results)
    #Let's write the dataframe to csv for later use
    dataloc.to_csv(r'data_{}_{}.csv'.format(lat,lon), index = False, header=True)
    return results
  
#Loop function to retrieve restaurants for all areas to evaluate
def get_restaurants(lats, lons):
    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to make sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        print(' .', end='')
        print(venues)
    print(' done.')
    return len(venues)



In [0]:
#Foursquare credentials 
foursquare_client_id = 'Key'
foursquare_client_secret = 'Key'

# Category IDs corresponding to Mediterranean/Moroccan restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
food_category = '4bf58dd8d48988d1c0941735' 

get_restaurants(latitudes, longitudes)

Obtaining venues around candidate locations:Empty DataFrame
Columns: []
Index: []
 .[]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


Empty DataFrame
Columns: []
Index: []
 .[]
Empty DataFrame
Columns: []
Index: []
 .[]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


                       referralId  ...  venue.photos.groups
0  e-0-4dcd6510c65bdac713466b0c-0  ...                   []

[1 rows x 19 columns]
 .[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4dcd6510c65bdac713466b0c', 'name': 'Thomas Mediteraan Restaurant', 'location': {'address': 'Krabbendijkestraat 95', 'lat': 51.872902, 'lng': 4.466133, 'labeledLatLngs': [{'label': 'display', 'lat': 51.872902, 'lng': 4.466133}], 'distance': 177, 'postalCode': '3086', 'cc': 'NL', 'city': 'Rotterdam', 'state': 'Zuid-Holland', 'country': 'Nederland', 'formattedAddress': ['Krabbendijkestraat 95', '3086 Rotterdam', 'Nederland']}, 'categories': [{'id': '4bf58dd8d48988d1c0941735', 'name': 'Mediterranean Restaurant', 'pluralName': 'Mediterranean Restaurants', 'shortName': 'Mediterranean', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_', 'suffix': '.png'}, 'primary': True}

...

Empty DataFrame
Columns: []
Index: []
 .[]
Empty DataFrame
Columns: []
Index: []
 .[]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


Empty DataFrame
Columns: []
Index: []
 .[]
Empty DataFrame
Columns: []
Index: []
 .[]
Empty DataFrame
Columns: []
Index: []
 .[]
 done.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


0

In [0]:
## DATA EXTRACTION; SKIP THIS PART IF YOU HAVE THE CSV FILE FOR THE RESTAURANTS ###
#let's load and merge the csv's to one dataframe
import os
import pandas as pd
import glob

path = '/content' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
  if os.stat(filename).st_size > 1:
    #print(filename)
    dff = pd.read_csv(filename, index_col=None, header=0)
    li.append(dff)
df = pd.concat(li, axis=0, ignore_index=True)

#write results to csv
#df.to_csv(r'data_complete.csv', index = False, header=True)

In [0]:
### Continue here with loaded .csv file ###
#Load the Mediterranean/Moroccan restaurant data for the areas  
df_mres = pd.read_csv('data_complete.csv', index_col=None, header=0)

locations = df_mres[['venue.location.lat', 'venue.location.lng']]
loclist = locations.values.tolist()
#len(loclist)

df_mres.head(10)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address,venue.location.postalCode,venue.location.city,venue.location.state,venue.location.crossStreet,venue.venuePage.id,venue.location.neighborhood
0,e-0-4b8d5d9ff964a52057f732e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b8d5d9ff964a52057f732e3,Tekbir Döner,51.931725,4.482236,"[{'label': 'display', 'lat': 51.9317246, 'lng'...",314,NL,Nederland,['Nederland'],"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-4dbb007f93a08f9274995748-0,0,"[{'summary': 'This spot is popular', 'type': '...",4dbb007f93a08f9274995748,Ömür,51.914523,4.465676,"[{'label': 'display', 'lat': 51.91452299999999...",134,NL,Nederland,"['Nieuwe Binnenweg 152', '3015 BG Rotterdam', ...","[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],Nieuwe Binnenweg 152,3015 BG,Rotterdam,Zuid-Holland,NaN,NaN,NaN
2,e-0-532dcb75498effc44e1b513a-1,0,"[{'summary': 'This spot is popular', 'type': '...",532dcb75498effc44e1b513a,Ten To Ten,51.916431,4.469877,"[{'label': 'display', 'lat': 51.916431, 'lng':...",296,NL,Nederland,"['Nieuwe Binnenweg 65 A', '3014 GD Rotterdam',...","[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],Nieuwe Binnenweg 65 A,3014 GD,Rotterdam,Zuid-Holland,NaN,NaN,NaN
3,e-0-4fd7609a7bebbea7ef448c8e-2,0,"[{'summary': 'This spot is popular', 'type': '...",4fd7609a7bebbea7ef448c8e,La Cantine & Haute Friture,51.913204,4.472272,"[{'label': 'display', 'lat': 51.91320350363546...",341,NL,Nederland,"['Museumpark', '3015 CB Rotterdam', 'Nederland']","[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],Museumpark,3015 CB,Rotterdam,Zuid-Holland,NaN,NaN,NaN
4,e-0-4bdb604363c5c9b642a92768-0,0,"[{'summary': 'This spot is popular', 'type': '...",4bdb604363c5c9b642a92768,Cafe Restaurant 't Centrum,51.920952,4.407998,"[{'label': 'display', 'lat': 51.92095180653114...",220,NL,Nederland,"['Stationsplein 1', '3112 HJ Schiedam', 'Neder...","[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],Stationsplein 1,3112 HJ,Schiedam,Zuid-Holland,NaN,NaN,NaN
5,e-0-5c2f628eacb37f002c4c4f0b-0,0,"[{'summary': 'This spot is popular', 'type': '...",5c2f628eacb37f002c4c4f0b,Bar Pulpo,51.905760,4.488081,"[{'label': 'display', 'lat': 51.90576, 'lng': ...",120,NL,Nederland,"['3072 AR Rotterdam', 'Nederland']","[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],NaN,3072 AR,Rotterdam,Zuid-Holland,NaN,NaN,NaN
6,e-0-5367fa29498efa709e9bb165-0,0,"[{'summary': 'This spot is popular', 'type': '...",5367fa29498efa709e9bb165,beyaz saray,51.933617,4.427559,"[{'label': 'display', 'lat': 51.933617, 'lng':...",278,NL,Nederland,['Rotterdamse Rijweg 128 (Duivesteijn Straat)'...,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],Rotterdamse Rijweg 128,NaN,NaN,NaN,Duivesteijn Straat,NaN,NaN
7,e-0-4d790b9d4b30a143c88a9dcb-1,0,"[{'summary': 'This spot is popular', 'type': '...",4d790b9d4b30a143c88a9dcb,FP Markt,51.935057,4.428039,"[{'label': 'display', 'lat': 51.93505722485189...",348,NL,Nederland,"['Burgemeester baumannlaan 164 (Baanweg)', 'Ro...","[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],Burgemeester baumannlaan 164,NaN,Rotterdam,Zuid-Holland,Baanweg,NaN,NaN
8,e-0-5807aa5338fa9f96dff60062-0,0,"[{'summary': 'This spot is popular', 'type': '...",5807aa5338fa9f96dff60062,Ayla,51.921200,4.471302,"[{'label': 'display', 'lat': 51.92119992018750...",229,NL,Nederland,"['Kruisplein 153', '3014 DD', 'Nederland']","[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],Kruisplein 153,3014 DD,NaN,NaN,NaN,NaN,NaN
9,e-0-4c6c1e5d46353704727f05bc-0,0,"[{'summary': 'This spot is popular', 'type': '...",4c6c1e5d46353704727f05bc,Grillroom Hadarom,51.898307,4.511255,"[{'label': 'display', 'lat': 51.89830743333333...",285,NL,Nederland,['Neder

In [0]:
#Plot the restaurants per area
import folium

map_rdam = folium.Map(location=rdam_center, zoom_start=12)
folium.Marker(rdam_center, popup='Coolsingel').add_to(map_rdam)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='red', fill=True, fill_color='green', fill_opacity=1).add_to(map_rdam) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_rdam)
    #folium.Marker([lat, lon]).add_to(map_rdam)
for point in range(0, len(loclist)):
    folium.Marker(loclist[point], popup=df_mres['venue.name'][point]).add_to(map_rdam)
map_rdam



Now we have all the restaurants in area within few kilometers from Coolsingel, and we know which ones are Medditerean/Moroccan restaurants for each designated area! 

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Medditerean/Moroccan restaurant.


# Methodology

In order to fully grasp the distribution of Medditerean/Moroccan restaurants in the city of Rotterdam, exploratory data analysis is required from the data obtained. The following subsections discuss the areas of interest in Rotterdam taking into account the city center (Coolsingel) and the location of the restaurants in Rotterdam.  


# Exploratory Data Analysis

Let's perform exploratory data analysis and derive additional insights from the data. First let's count the number of Medditerean/Moroccan restaurants in every area candidate. 

In [0]:
#Analyses to do
#1. define number of Medditerean/Moroccan restaurants
print('List of Medditerean/Moroccan restaurants')
print('---------------------------')
print('Total:', len(df_mres))

#2. define the distance from Coolsingel (farest versus nearest; how many are within 1km? 3km? etc)
#3. Define possible clusters of restaurants and are there any significant area's with a lot of restaurants
#4. Note that in alexander there are NO medditerean restaurant -> further research for potential area to start one



List of Medditerean/Moroccan restaurants
---------------------------
Total: 63


After defining the candidate areas, the next task is to retrieve and investigate the distribution of the location of the Medditerean/Moroccan restaurants. Important here is to determine which candidate areas include restaurants around the city center. FIgure 4 shows the distribution of the restaurants in the candidate areas around the city center. 

In [0]:
#Prep a cleaned dataframe for further analysis
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

In [0]:
df_mres.head(1)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address,venue.location.postalCode,venue.location.city,venue.location.state,venue.location.crossStreet,venue.venuePage.id,venue.location.neighborhood
0,e-0-4b8d5d9ff964a52057f732e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b8d5d9ff964a52057f732e3,Tekbir Döner,51.931725,4.482236,"[{'label': 'display', 'lat': 51.9317246, 'lng'...",314,NL,Nederland,['Nederland'],"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
dff = pd.DataFrame({'Address': df_mres['venue.location.address'],
                    'ResName': df_mres['venue.name'],
                    'Latitude': df_mres['venue.location.lat'],
                    'Longitude': df_mres['venue.location.lng']})

In [0]:
#Prep a dataframe for each retrieved restaurant 
dff.head(10)

,Address,ResName,Latitude,Longitude
0,NaN,Tekbir Döner,51.931725,4.482236
1,Nieuwe Binnenweg 152,Ömür,51.914523,4.465676
2,Nieuwe Binnenweg 65 A,Ten To Ten,51.916431,4.469877
3,Museumpark,La Cantine & Haute Friture,51.913204,4.472272
4,Stationsplein 1,Cafe Restaurant 't Centrum,51.920952,4.407998
5,NaN,Bar Pulpo,51.905760,4.488081
6,Rotterdamse Rijweg 128,beyaz saray,51.933617,4.427559
7,Burgemeester baumannlaan 164,FP Markt,51.935057,4.428039
8,Kruisplein 153,Ayla,51.921200,4.471302
9,NaN,Grillroom Hadarom,51.898307,4.511255


In [0]:
#Determine the Xs and Ys and distance of restaurant to rdam center
res_distances_from_center = []
xRes = []
yRes = []

for i in range(len(dff)):
  x, y = lonlat_to_xy(dff['Longitude'][i], dff['Latitude'][i])
  distance_from_center = calc_xy_distance(rdam_center_x, rdam_center_y, x, y)
  xRes.append(x)
  yRes.append(y)
  res_distances_from_center.append(distance_from_center)
  

In [0]:
dff['X'] = xRes
dff['Y'] = yRes
dff['Distance to Rdam center'] = res_distances_from_center

In [0]:
dff.head(10)

,Address,ResName,Latitude,Longitude,X,Y,Distance to Rdam center
0,NaN,Tekbir Döner,51.931725,4.482236,-222157.284492,5.805888e+06,1088.107007
1,Nieuwe Binnenweg 152,Ömür,51.914523,4.465676,-223569.996219,5.804148e+06,1253.050568
2,Nieuwe Binnenweg 65 A,Ten To Ten,51.916431,4.469877,-223251.322890,5.804318e+06,896.323174
3,Museumpark,La Cantine & Haute Friture,51.913204,4.472272,-223139.548859,5.803936e+06,1107.550807
4,Stationsplein 1,Cafe Restaurant 't Centrum,51.920952,4.407998,-227415.665964,5.805440e+06,4901.949821
5,NaN,Bar Pulpo,51.905760,4.488081,-222177.110878,5.802953e+06,1953.586778
6,Rotterdamse Rijweg 128,beyaz saray,51.933617,4.427559,-225870.013237,5.806646e+06,3767.184802
7,Burgemeester baumannlaan 164,FP Markt,51.935057,4.428039,-225813.679450,5.806801e+06,3793.839687
8,Kruisplein 153,Ayla,51.921200,4.471302,-223076.462280,5.804832e+06,531.188297
9,NaN,Grillroom Hadarom,51.898307,4.511255,-220709.736332,5.801896e+06,3496.773972


# Modeling

**Identify clusters**


In the previous part, the restaurants retrieved were plotted on the map within the candidate areas. The next step is to identify the restaurant density across the candidate areas of Rotterdam. In order to do so accordingly, a heatmap is created to optically identify  promising candidate areas close to the city center with low number of Medditerean/Moroccan restaurants. Note that the higher the density of the number of restaurants within a certain candidate area, the warmer the color.

In [0]:
#plot values for restaurants
import matplotlib.pyplot as plt
plt.scatter(dff['X'], dff['Y'])
plt.show()

In [0]:
import requests
import folium
from folium import plugins
from folium.plugins import HeatMap

resLatitudes = dff['Latitude']
resLongitudes = dff['Longitude']

restaurant_latlons = [[lat, lon] for lat, lon in zip(resLatitudes, resLongitudes)]


rdam_boroughs_url = 'https://raw.githubusercontent.com/blackmad/neighborhoods/master/gn-rotterdam.geojson'
rdam_boroughs = requests.get(rdam_boroughs_url).json()

def borough_style(feature):
  return {'color': 'blue', 'fill': False}


map_rdam = folium.Map(location=rdam_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_rdam)
HeatMap(restaurant_latlons).add_to(map_rdam)
#folium.Circle(rdam_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_rdam)
folium.Marker(rdam_center).add_to(map_rdam)
for lat, lon in zip(resLatitudes, resLongitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rdam) 
folium.GeoJson(rdam_boroughs, style_function=borough_style, name='geojson').add_to(map_rdam)
map_rdam

**K-means clustering**

The heatmap suggests the existence of clusters around the city center of Rotterdam. By applying the k-means clustering algorithm, an attempt is made to determine the existence of clusters. I have chosen to take k=15 as the number of possible clusters to take into consideration. After determining the clusters, a plot on the heat map is made to optically identify the clusters. Indeed, there are a significant number of clusters around the city center with groups of restaurants. Indeed, one should consider the area of clusters with a minimum and maximum number of corresponding restaurants to start one.

In [0]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = dff[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_rdam = folium.Map(location=rdam_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_rdam)
HeatMap(restaurant_latlons).add_to(map_rdam)
folium.Circle(rdam_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_rdam)
folium.Marker(rdam_center).add_to(map_rdam)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_rdam) 
for lat, lon in zip(resLatitudes, resLongitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rdam)
folium.GeoJson(rdam_boroughs, style_function=borough_style, name='geojson').add_to(map_rdam)
map_rdam


# Conclusion
The purpose of this project was to identify Meditterean/Moroccan restaurants in  candidate areas from the city center of Rotterdam and identify areas with low density of restaurants  in order to aid stakeholders in narrowing down the search for optimal location for a new business venture. The overall analysis shows that although there is a great number of restaurants in Rotterdam, there are interesting areas of low restaurant density fairly close to the city center as seen in the heatmap.The highest concentration of restaurants is clearly in areas close by the city center. This is explainable due to the fact that the city center is the place masses gather. The remaining interesting clusters can be explained by a combination of popularity among tourists, closeness to the city center and strong socio-economic dynamics. Nonetheless, the clusters show a lower restaurant density. From there, these areas are recommended for potential business ventures and should therefore be considered as a starting point for more detailed analysis. 


# Future work 
Indeed, one should consider the area of clusters with a minimum number of corresponding restaurants and high potentials within the candidate area. In order to come to a final decision on an optimal restaurant location, further research is essential. Based on specific characteristics of neighborhoods and locations in every recommended area, investors have to take into account additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc. Thus, further analysis is recommended per designated candidate area with the appropriate level of restaurant density. 

